In [1]:
'''
Date: 2022.07.29
Title: 데이터 분류
By: Kang Jin Seong
'''
# -*- coding: utf-8 -*-

%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

'''
감성 분류: 문서(텍스트 데이터)를 긍정의견 또는 부정의견으로 나누어 분류하는것

'''

'\n감성 분류: 문서(텍스트 데이터)를 긍정의견 또는 부정의견으로 나누어 분류하는것\n\n'

In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time

# 윈도우용 크롬 웹드라이버 실행 경로
excutable_path = 'C:/Users/USER/DataAnalysis_jupyter/chromedriver.exe'

#크롤링할 사이트 주소를 정의합니다.
source_url = 'https://map.kakao.com/'

#크롬 드라이버를 사용합니다.
driver = webdriver.Chrome(executable_path = excutable_path)

#카카오 지도에 접속합니다.
driver.get(source_url)

# 검색창에 검색어를 입력합니다.
searchbox = driver.find_element('xpath',"//input[@id= 'search.keyword.query']")
searchbox.send_keys('강남역 맛집')

# 검색버튼을 눌러서 결과를 가져옵니다.
searchbutton = driver.find_element('xpath', "//button[@id = 'search.keyword.submit']")
driver.execute_script('arguments[0].click();', searchbutton)

# 검색 결과를 가져올 시간을 기다립니다.
time.sleep(2)

# 검색 결과의 페이지 소스를 가져옵니다.
html = driver.page_source

# BeautifulSoup을 이용하여 html 정보를 파싱합니다.
soup = BeautifulSoup(html, 'html.parser')
moreviews = soup.find_all(name = 'a', attrs = {'class':'moreview'})

# a 태그의 href 속성을 리스트로 추출하여, 크롤링 할 페이지 리스트를 생성합니다.
page_urls = []

for moreview in moreviews:
    page_url = moreview.get('href')
    print(page_url)
    page_urls.append(page_url)
    
# 크롤링에 사용한 브라우저를 종료합니다.
driver.close()

https://place.map.kakao.com/85570955
https://place.map.kakao.com/26431943
https://place.map.kakao.com/27270313
https://place.map.kakao.com/1238400864
https://place.map.kakao.com/25891059
https://place.map.kakao.com/1752978029
https://place.map.kakao.com/1863534623
https://place.map.kakao.com/109169564
https://place.map.kakao.com/95713992
https://place.map.kakao.com/25036974
https://place.map.kakao.com/13575898
https://place.map.kakao.com/2011092566
https://place.map.kakao.com/27564595
https://place.map.kakao.com/1465968863
https://place.map.kakao.com/1503746075
https://place.map.kakao.com/741391811
https://place.map.kakao.com/495658881
https://place.map.kakao.com/27504403


In [3]:
'''
가져온 맛집 리스트의 리뷰 정보 크롤링하기
'''

columns = ['score', 'review']
df = pd.DataFrame(columns = columns)

driver = webdriver.Chrome(executable_path = excutable_path)

for page_url in page_urls:
    
    # 상세 보기 페이지에 접속합니다.
    driver.get(page_url)
    time.sleep(2)
    
    # 첫 페이지 리뷰를 크롤링합니다.
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    contents_div = soup.find(name = 'div', attrs = {'class':'evaluation_review'})
#     print(contents_div)
    # 별점을 가져옵니다.

#     rates = contents_div.find_all(name = 'em', attrs = {"class": "num_rate"})
    rates = contents_div.find_all(name = 'span', attrs = {"class": "txt_desc"})
    # 리뷰를 가져옵니다.
    reviews = contents_div.find_all(name = 'p', attrs = {"class": "txt_comment"})
    
    for rate, review in zip(rates, reviews):
        row = [rate.text[0], review.find(name = 'span').text]
        series = pd.Series(row, index = df.columns)
        df = df.append(series, ignore_index = True)
#         print(df)
        
    # 2-5페이지의 리뷰를 크롤링합니다.
    for button_num in range(2,6):
        # 오류가 나는 경우 (리뷰 페이지가 없는 경우) 수행하지 않습니다.
        
        try:
            another_reviews = driver.find_element('xpath', "//a[@data-page  = '" + str(button_num) + "']")
            another_reviews.click()
            time.sleep(2)
            
            # 페이지 리뷰를 크롱링합니다.
            
            html = driverr.page_source
            soup = BeautifulSoup(html, 'html.parser')
            contents_div = soup.find(name = 'div', attrs = {"class": "evaluation_review"})
            
            # 별점을 가져옵니다.
#             rates = contents_div.find_all(name = 'em', attrs = {"class": "num_rate"})
            rates = contents_div.find_all(name = 'span', attrs = {"class": "txt_desc"})
            
            # 리뷰를 가져옵니다.
            reviews = contents_div.find_all(name = 'p', attrs = {"class": "txt_comment"})
            
            for rate, review in zip(rates, reviews):
                row = [rate.text[0], review.find(name = 'span').text]
                series = pd.Series(row, index = df.columns)
                df = df.append(series, ignore_index = True)
#                 print(df)
                
        except:
#             print('end')
            break
driver.close()

AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
# 4점 이상의 리뷰는 긍정, 3점 이하의 리뷰는 부정으로 평가합니다.
df['y'] = df['score'].apply(lambda x: 1 if float(x) > 3 else 0)
print(df.shape)
df.head()

In [ ]:
df.to_csv("review_data.csv", index = False)

In [ ]:
'''
TF-IDF를 이용한 핵심어 추출
 - 형태소 추출하기(한글 텍스트로 전처리)
'''
df = pd.read_csv("review_data.csv")

import re

# 텍스트 정제 함수 : 한글 이외의 문자는 전부 제거
def text_cleaning(text):
    # 한글의 정규표현식으로 한글만 추출합니다.
    hangul = re.compile('[^ ㄱ-ㅣ가-힣]+')
    result = hangul.sub('', str(text))
    return result

In [ ]:
# 함수를 적용하여 리뷰에서 한글만 추출합니다.
df = pd.read_csv("review_data.csv")
df['ko_text'] = df['review'].apply(lambda x: text_cleaning(x))
del df['review']

# 한 글자 이상의 텍스트를 가지고 있는 데이터만 추출합니다.
df = df[df['ko_text'].str.len() > 0]
df.head()

In [ ]:
'''형태소 단위로 추출'''

from konlpy.tag import Okt

# konlpy 라이브러리로 텍스트 데이터에서 형태소를 추출합니다.
def get_pos(x):
    tagger = Okt()
    pos = tagger.pos(x)
    pos = ['{}/{}'.format(word, tag) for word, tag in pos]
    return pos

# 형태소 추출 동작을 테스트 합니다.
result = get_pos(df['ko_text'].values[0])
print(result)

In [ ]:
'''
Date: 2022.08.08

- 분류 모델의 학습 데이터로 변환하기
  - raw 데이터셋: 데이터 프레임의 텍스트 데이터에 해당합니다.
  - 말뭉치: raw 데이터셋으로부터 말뭉치를 생성합니다. 이 말뭉치는 형태소의 서로 다른 고유 한 셋을 가지고 있습니다.
  - 학습 데이터 셋: 서로다른 형태소는 각 텍스트 데이터의 벡터 길이가 됩니다.
                    만약 텍스트에 해당 단어가 존재하면 벡터의 값을 1로, 존재하지 않으면 벡터의 값을 0으로 할당합니다.(p242 참조)

'''
# corpus index 생성
from sklearn.feature_extraction.text import CountVectorizer

# 형태소를 벡터 형태의 학습 데이터 셋(X데이터)으로 변환합니다.
index_vectorizer = CountVectorizer(tokenizer = lambda x: get_pos(x))
X = index_vectorizer.fit_transform(df['ko_text'].tolist())

In [ ]:
X.shape

In [ ]:
print(str(index_vectorizer.vocabulary_)[:100]+"..")

In [ ]:
print(df['ko_text'].values[0])
print(X[0])

In [ ]:
# TF-IDF(Term Frequency - Inverse Documnet Frequency)
# 단어의 빈도를 나타내는 TF 와 문서 빈도를 나타내는 DF의 역수인 IDF를 곱한 값을 의미힌다.
# 이는 다른 문서들에서는 등장하지 않았지만 현재 문서에서는 많이 등장하는 단어를 의미하며 그 단어가 현재 문서에서 얼마나 중요한지를
# 피처로 나타낼수 있는 방법입니다.

from sklearn.feature_extraction.text import TfidfTransformer

# TF-IDF 방법으로 형태소를 벡터 형태의 학습 데이터셋(X 데이터)으로 변환합니다.
tfidf_vectorizer = TfidfTransformer()
X = tfidf_vectorizer.fit_transform(X)

In [ ]:
print(X.shape)
print(X[0])

In [ ]:
'''긍정 부정 리뷰 분류하기'''

# 분류 모델링(데이터셋 분리)

from sklearn.model_selection import train_test_split

y = df['y']
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size = 0.30)
print(x_train.shape)
print(x_test.shape)

In [ ]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# 로지스틱 회귀 모델을 학습합니다.
lr = LogisticRegression(random_state = 0)
lr.fit(x_train, y_train)
y_pred = lr.predict(x_test)
y_pred_probability = lr.predict_proba(x_test)[:,1]

# 로지스틱 회귀모델의 성능을 평가합니다.
print("accuracy: %.2f" % accuracy_score(y_test, y_pred))
print("Precision: %.3f" % precision_score(y_test,y_pred))
print("Recall: %.3f" % recall_score(y_test, y_pred))
print("F1: %.3f" % f1_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import confusion_matrix

# Confusion Matrix를 출력합니다.
confmat = confusion_matrix(y_true=y_test, y_pred=y_pred)
print(confmat)

In [ ]:
# 클래스 불균형 문제 해결하기

# y가 0과 1을 각각 얼마나 가지고 있는지를 출력합니다.
df['y'].value_counts()

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

# AUC를 계산합니다.
false_positive_rate, true_positive_rate, thresholds = roc_curve(y_test, y_pred_probability)
roc_auc = roc_auc_score(y_test, y_pred_probability)
print("AUC: %.3f" % roc_auc)

# ROC Curve 그래프를 출력합니다.
plt.rcParams['figure.figsize'] = [5,4]
plt.plot(false_positive_rate, true_positive_rate, label = "ROC curve (area = %0.3f)" % roc_auc, color= 'red', linewidth = 4.0)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve of Logistic regression')
plt.legend(loc="lower right")

In [ ]:
'''중요 키워드 분석하기'''

# 회귀 모델의 피처 영향력 추출
# 학습한 회귀 모델의 계수를 출력합니다.
plt.rcParams['figure.figsize'] = [10,8]
plt.bar(range(len(lr.coef_[0])), lr.coef_[0])

In [ ]:
print(sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = True)[:5])
print(sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = True)[-5:])

In [ ]:
# 중요 피처의 형태소
# 회귀 모델의 계수를 높은 순으로 정렬합니다.
coef_pos_index = sorted(((value, index) for index, value in enumerate(lr.coef_[0])), reverse = True)

In [ ]:
# 회귀 모델의 계수를 index_vectorize 에 맵핑하여, 어떤 형태소인지 출력 할 수 있게 합니다.
invert_index_vectorizer = {v: k for k, v in index_vectorizer.vocabulary_.items()}

# 계수가 높은 순으로, 피처에 형태소를 맵핑한 결과를 출력합니다. 계수가 높은 피처는 리뷰에 긍정적인 영향을 주는 형태소라고 할 수 있습니다.
print(str(invert_index_vectorizer)[:100] + '..')

In [ ]:
# 상위 20개 긍정 형태소를 출력합니다.
for coef in coef_pos_index[:20]:
    print(invert_index_vectorizer[coef[1]], coef[0])